In [1]:
import pandas as pd
import os
import numpy as np
from paths import path_data, path_out_data

In [2]:
file_train = "DTC_train_smile.csv"
file_descriptor_inchikey = "key_descriptor_train.npy"

data_train = pd.read_csv("%s/%s"%(path_out_data,file_train))
descriptor_inchikey = np.load("%s/descriptors/%s"%(path_out_data,file_descriptor_inchikey)).item()

data_train["descriptors"] = data_train.standard_inchi_key.map(descriptor_inchikey)

In [3]:
# mask rows without descriptors
mask = ~data_train.descriptors.isna()
data_complete = data_train[mask].loc[:,["standard_inchi_key","target_id","Kd","descriptors"]].copy()

In [4]:
train_expand = pd.concat([data_complete.drop(['descriptors'], axis=1), data_complete['descriptors'].apply(pd.Series)], axis=1)

In [5]:
#match testing data to descriptors
file_test = "round_1_template.csv"
data_test = pd.read_csv("%s/%s"%(path_data,file_test))

#check the overlapping between testing and training

inchikey_unique = set(train_expand.standard_inchi_key)
sum(data_test.Compound_InchiKeys.apply(lambda x: x in inchikey_unique))

protein_unique = set(train_expand.target_id)
sum(data_test.UniProt_Id.apply(lambda x: x in protein_unique))

In [6]:
descriptor_test = np.load("%s/descriptors/smile_descriptor_test.npy"%(path_out_data)).item()
#map SMILE to descriptors
data_test["descriptors"] = data_test.Compound_SMILES.map(descriptor_test)

In [7]:
# slice the three columns only
temp = data_test.loc[:,["Compound_InchiKeys","UniProt_Id","descriptors"]]
#expand the descriptor column
test_expand = pd.concat([temp.drop(['descriptors'], axis=1), temp['descriptors'].apply(pd.Series)], axis=1)
#rename the first two columns of testing data
test_expand.rename(columns={'Compound_InchiKeys':'standard_inchi_key',
                           'UniProt_Id':'target_id'}, inplace=True)

In [8]:
#merge train and test
train_test = pd.concat([train_expand,test_expand], axis = 0)

#move the special three columns to the left for easier index later
columns = ['Kd','standard_inchi_key','target_id']
train_test = pd.concat([train_test.loc[:,columns], train_test.drop(columns, axis=1)], axis=1)

In [9]:
train_test.head()

,Kd,standard_inchi_key,target_id,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,log_p
0,10000.000000,AAKJLRGGTJKAMG-UHFFFAOYSA-N,O00329,1.956917,1001.004202,20.476481,16.47465,16.47465,14.245045,9.1078,...,0,0,1,0,0,0,0,2,0,3.4051
1,10000.000000,AAKJLRGGTJKAMG-UHFFFAOYSA-N,O00750,1.956917,1001.004202,20.476481,16.47465,16.47465,14.245045,9.1078,...,0,0,1,0,0,0,0,2,0,3.4051
2,10000.000000,AAKJLRGGTJKAMG-UHFFFAOYSA-N,O14730,1.956917,1001.004202,20.476481,16.47465,16.47465,14.245045,9.1078,...,0,0,1,0,0,0,0,2,0,3.4051
3,10000.000000,AAKJLRGGTJKAMG-UHFFFAOYSA-N,O14920,1.956917,1001.004202,20.476481,16.47465,16.47465,14.245045,9.1078,...,0,0,1,0,0,0,0,2,0,3.4051
4,173.338571,AAKJLRGGTJKAMG-UHFFFAOYSA-N,O14976,1.956917,1001.004202,20.476481,16.47465,16.47465,14.245045,9.1078,...,0,0,1,0,0,0,0,2,0,3.4051


In [10]:
# check the number of constant column
sum(train_test.apply(lambda x: max(x)==min(x), axis =0))

24

In [11]:
# remove constant columns
train_test = train_test.loc[:,train_test.apply(pd.Series.nunique) != 1]

#save the train_test data
train_test.to_csv("%s/train_test.csv"%(path_out_data), index=False)

In [13]:
# to do: fill the missing features in Mauroen's feature (the testing features are missing)
file_mauroen = "Chem_descriptor_Marouen.csv"
feature_mauroen = pd.read_csv("%s/%s"%(path_out_data,file_mauroen))

In [14]:
keys_unique = set(feature_mauroen.iloc[:,0])
sum(train_test.standard_inchi_key.apply(lambda x: x in keys_unique))

17031

In [15]:
sum(test_expand.standard_inchi_key.apply(lambda x: x in keys_unique))

19

In [16]:
test_expand.shape

(430, 200)